# Validation - stellar evolution

Generates the plots presented in SynthPop Paper 1, Section 6.3

Macy Huston & Jonas Klüter

In [ ]:
# Imports
import sys
import os
import numpy as np
import pandas
import matplotlib.pyplot as plt
import pandas as pd
from synthpop.modules.evolution.mist import MIST
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

columns = ["EEP","UBVRIplus", "log10_isochrone_age_yr", "initial_mass", "[Fe/H]_init", "phase", 'star_mass']
mist = MIST(columns)
mist4 = mist.read_csv("mist_web_interpolator/pop4.cmd")
mist5 = mist.read_csv("mist_web_interpolator/pop5.cmd")
mist6 = mist.read_csv("mist_web_interpolator/pop6.cmd")

import synthpop

def get_color_iter(n):
    return iter(['red', 'green', 'blue', 'gold'])

In [ ]:
# SynthPop Configuration
config = {
    "name_for_output":"validation_model",
    "model_name": "validation_model",
    "l_set": [0,90, 12.17, 39.14],
    "max_distance":25,
    "b_set":[0, 0, 5.37, 8.53],
    "l_set_type":"pairs",
    "b_set_type":"pairs",
    "solid_angle":[1e-4,3e-2,1e-3, 1e-2],
    "extinction_map_kwargs": {
        "name":"MapsFromDustmaps", "dustmap_name":"marshall"
        },
    "output_file_type":"hdf5",
    "obsmag": False,
    "chosen_bands": ["Bessell_B", "Bessell_V", "Bessell_I"],
    "maglim": ["Bessell_V", 99999, "keep"],
    "lost_mass_option": 1,
    "overwrite":True
}

# set up SynthPop model
model = synthpop.SynthPop(**config)
# initialize populations
model.init_populations()

In [ ]:
# SynthPop Catalog Generation

# place to collect the data from different line of sights
data = {}
# loop over all location
for i,loc in enumerate(model.get_iter_loc()):
    # run synthpop for the given location and solid angle
    data_loc, distribution = model.process_location(
        *loc, model.parms.solid_angle[i], model.parms.solid_angle_unit,
        save_data=False)

    # store data in the dictionary
    data[loc] = data_loc
combined_data = pandas.concat([d for d in data.values()])

In [ ]:
ccycle = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']

In [ ]:
# Collect the stars for each population we are examining
pop4 = combined_data[combined_data['pop']==4]
pop5 = combined_data[combined_data['pop']==5]
pop6 = combined_data[combined_data['pop']==6]

In [ ]:
# Plot the CMD of our stars and the MIST web interpolator isochrones
plt.figure(figsize=(6.5,6.5))
plt.plot(mist4.Bessell_B - mist4.Bessell_V, mist4.Bessell_V, c=ccycle[0], linewidth=6, alpha=0.25, label='[Fe/H]=-0.50 dex, Age=10.00 Gyr')
plt.plot(pop4.Bessell_B - pop4.Bessell_V, pop4.Bessell_V, c=ccycle[0], marker='.', linestyle='none', markersize=2)
plt.plot(mist5.Bessell_B - mist5.Bessell_V, mist5.Bessell_V, c=ccycle[1], linewidth=6, alpha=0.25, label='[Fe/H]=-0.32 dex, Age=9.135 Gyr')
plt.plot(pop5.Bessell_B - pop5.Bessell_V, pop5.Bessell_V, c=ccycle[1], marker='.', linestyle='none', markersize=2)
plt.plot(mist6.Bessell_B - mist6.Bessell_V, mist6.Bessell_V, c=ccycle[2], linewidth=6, alpha=0.25, label='[Fe/H]=-1.14 dex, Age=4.230 Gyr')
plt.plot(pop6.Bessell_B - pop6.Bessell_V, pop6.Bessell_V, c=ccycle[2], marker='.', linestyle='none', markersize=2)
plt.legend(loc=4)
plt.xlabel('B-V [mag.]'); plt.ylabel('V [mag.]')
plt.ylim(20,-5.5)
plt.tight_layout()
plt.savefig('validation_figures/evolution_cmd.png')